In [438]:
import numpy as np
import math
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from bs4 import BeautifulSoup
import re
import requests
import unidecode

pd.set_option('display.max_columns', None)

pd.options.display.max_columns=999

import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [439]:
PlayerGameData = pd.read_csv('../Data/FBRef_MLS_PlayerGames.csv')
PlayerGameData['Season'] = PlayerGameData['Date'].apply(lambda x: int(str(x)[:4]))
PlayerGameData = PlayerGameData.sort_values(by='Date', ascending=False).reset_index(drop=True)
PlayerGameData

,Date,Team,Opp,HorA,Manager,Captain,Referee,Player,#,Nation,Pos,Age,Min,Sub,Replace,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%,Season
0,20211211,Portland Timbers,New York City FC,H,Giovanni Savarese,Diego Chará,"Providence Park, Portland, OR",Claudio Bravo,5,ARG,LB,24.758333,120,0,NaN,0,0,0,0,2,0,0,0,NaN,24,7,5,2,0.0,0.0,0.3,4,0,71,101,70.3,9,71,9,1,3,71,101,70.3,1398,697,30,38,78.9,32,43,74.4,9,19,47.4,0,0.3,3,6,7,3,9,80,21,3,0,13,4,5,1,0,1,0,48,19,34,65,10,7,17,0,71,0,2,4,4,7,4,4,3,0,1,4,25.0,3,24,8,33.3,15,6,3,2,0,0,2,5,12,3,0,125,7,44,51,37,0,104,1,3,33.3,2,0,71,348,200,9,2,0,4,0,62,58,93.5,2,0,0,0,3,1,0,5,5,4,0.0,0.0,0,23,2,3,40.0,2021
1,202112

In [440]:
# This is tough
# FBRef can list positions in games as 'WB,LB,LM' as well as 'LB'
# We want to pick out ones in which a wing position is present
# Have to iterate through, split on the comma and check for a wing position

wb_pos_list = ['LB', 'LM', 'LW', 'RB', 'RM', 'RW', 'WB']
to_keep = []
for i in PlayerGameData.index:
    pos = PlayerGameData['Pos'][i]
    if (pos in wb_pos_list):
        to_keep.append(i)
    elif (len(pos.split(',')) > 1):
        for p in pos.split(','):
            if (p not in wb_pos_list):
                break
        to_keep.append(i)
        
Wing_GameData = PlayerGameData.loc[to_keep].reset_index(drop=True)
Wing_GameData

,Date,Team,Opp,HorA,Manager,Captain,Referee,Player,#,Nation,Pos,Age,Min,Sub,Replace,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%,Season
0,20211211,Portland Timbers,New York City FC,H,Giovanni Savarese,Diego Chará,"Providence Park, Portland, OR",Claudio Bravo,5,ARG,LB,24.758333,120,0,NaN,0,0,0,0,2,0,0,0,NaN,24,7,5,2,0.0,0.0,0.3,4,0,71,101,70.3,9,71,9,1,3,71,101,70.3,1398,697,30,38,78.9,32,43,74.4,9,19,47.4,0,0.3,3,6,7,3,9,80,21,3,0,13,4,5,1,0,1,0,48,19,34,65,10,7,17,0,71,0,2,4,4,7,4,4,3,0,1,4,25.0,3,24,8,33.3,15,6,3,2,0,0,2,5,12,3,0,125,7,44,51,37,0,104,1,3,33.3,2,0,71,348,200,9,2,0,4,0,62,58,93.5,2,0,0,0,3,1,0,5,5,4,0.0,0.0,0,23,2,3,40.0,2021
1,202112

In [441]:
# Group by player, but keep individual seasons separate
Wing_PlayerGrouped = Wing_GameData.groupby(by=['Player', 'Season']).sum().reset_index()
Wing_PlayerGrouped = Wing_PlayerGrouped[Wing_PlayerGrouped['Min'] >= 720]
Wing_PlayerGrouped

,Player,Season,Date,#,Age,Min,Sub,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%
0,A.J. DeLaGarza,2019,464383463,460,726.686111,1991,0,0,1,0,0,6,1,5,0,0.0,293,43,17,32,0.2,0.2,0.5,14,1,842,1021,1873.0,77,663,51,6,9,842,1021,1873.0,15998,5417,332,360,2127.7,393,454,1966.0,108,188,1295.6,1,0.5,7,45,6,2,77,798,223,27,1,130,18,17,0,0,0,0,580,209,232,95,650,62,196,5,842,3,18,16,11,43,21,23,17,3,18,39,833.4,21,293,61,435.1,159,111,23,32,10,0,22,17,60,64,0,1202,69,397,607,265,17,985,6,9,400.0,6,1,663,3198,1544,51,20,4,9,8,706,662,2142.4,29,5,0,0,27,26,1,17,17,21,0.0,1.0,0,163,18,21,858.3
3,Aaron Herrera,2019,686482257,748,750.277778,2948,1,0,3,0,0,19,6,7,1,0.0

In [442]:
### Stats to look at: ###
# 1v1 Attacking
#   - Possession_Dribbles_Succ / Possession_Dribbles_Att
#   - Possession_Carries_PrgDist
# 1v1 Defending
#   - Defensive Actions_Vs Dribbles_Tkl / Defensive Actions_Vs Dribbles_Att (% is lost in groupby().sum()) (scale dot in swarm by volume?)
# Crossing
#   - Passing__CrsPA
#   - Passing__CrsPA / Pass Types_Pass Types_Crs  (% of crosses completed in the penalty area)
# Fitness
#   - Defensive Actions_Pressures_Press  (Rough estimate of work rate - Would be nice to see throughout 90 to tell if it drops off)
#   - % of games completed (full 90)
# Variety of Positions
#   - Viz to show breakdown of minutes by position
#   - Total measure - % Minutes played at secondary positions

In [443]:
# Combine seasons now for if I need a single player entry encompassing 2019-2021
AllYears = Wing_PlayerGrouped.groupby(by='Player').sum().reset_index()
AllYears

,Player,Season,Date,#,Age,Min,Sub,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%
0,A.J. DeLaGarza,2019,464383463,460,726.686111,1991,0,0,1,0,0,6,1,5,0,0.0,293,43,17,32,0.2,0.2,0.5,14,1,842,1021,1873.0,77,663,51,6,9,842,1021,1873.0,15998,5417,332,360,2127.7,393,454,1966.0,108,188,1295.6,1,0.5,7,45,6,2,77,798,223,27,1,130,18,17,0,0,0,0,580,209,232,95,650,62,196,5,842,3,18,16,11,43,21,23,17,3,18,39,833.4,21,293,61,435.1,159,111,23,32,10,0,22,17,60,64,0,1202,69,397,607,265,17,985,6,9,400.0,6,1,663,3198,1544,51,20,4,9,8,706,662,2142.4,29,5,0,0,27,26,1,17,17,21,0.0,1.0,0,163,18,21,858.3
1,Aaron Herrera,6060,1656391459,1804,1890.930556,7180,1,1,12,0,0,55,13,23

In [444]:
# Offensive Progressive Dribbling Ability
AllYears['PrgDist_p90'] = AllYears['Possession_Carries_PrgDist']/(AllYears['Min']/90)

# Offensive 1v1 Ability
AllYears['Possession_Dribbles_Succ%'] = (AllYears['Possession_Dribbles_Succ'] / AllYears['Possession_Dribbles_Att'])*100
AllYears['Dribbles_Att_p90'] = AllYears['Possession_Dribbles_Att']/(AllYears['Min']/90)

# Defensive 1v1 Ability
AllYears['Defensive Actions_Vs Dribbles_Tkl%'] = (AllYears['Defensive Actions_Vs Dribbles_Tkl'] / AllYears['Defensive Actions_Vs Dribbles_Att'])*100
AllYears['1v1_Tkls_p90'] = AllYears['Defensive Actions_Vs Dribbles_Att']/(AllYears['Min']/90)

# Crossing Ability
AllYears['CrsPA%'] = (AllYears['Passing__CrsPA'] / AllYears['Pass Types_Pass Types_Crs'])*100
AllYears['Crs_p90'] = AllYears['Pass Types_Pass Types_Crs']/(AllYears['Min']/90)

# Fitness
AllYears['Press_p90'] = AllYears['Defensive Actions_Pressures_Press']/(AllYears['Min']/90)

target_cols = ['Player', 'Min', 'PrgDist_p90', 'Possession_Dribbles_Succ', 'Possession_Dribbles_Att', 'Dribbles_Att_p90', 'Possession_Dribbles_Succ%',
               'Defensive Actions_Vs Dribbles_Tkl', 'Defensive Actions_Vs Dribbles_Att', '1v1_Tkls_p90', 'Defensive Actions_Vs Dribbles_Tkl%',
               'Crs_p90', 'CrsPA%', 'Press_p90']
AllYears_target = AllYears[(AllYears['Dribbles_Att_p90'] >= 0.5) & (AllYears['1v1_Tkls_p90'] >= 0.5) & (AllYears['Crs_p90'] >= 0.5)].sort_values(by='Possession_Dribbles_Succ%', ascending=False)[target_cols]
AllYears_target

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90
122,Javain Brown,1751,72.575671,14,17,0.873786,82.352941,32,68,3.495146,47.058824,2.826956,18.181818,18.246716
87,Emmanuel Más,773,124.695990,9,11,1.280724,81.818182,9,19,2.212160,47.368421,1.979301,11.764706,13.040103
222,Paxton Pomykal,1422,124.240506,16,20,1.265823,80.000000,19,38,2.405063,50.000000,2.341772,10.810811,21.012658
83,Ema Twumasi,1702,144.095182,31,39,2.062280,79.487179,18,36,1.903643,50.000000,1.533490,13.793103,14.488837
147,Jorge Moreira,2319,217.335058,31,39,1.513583,79.487179,39,81,3.143596,48.148148,4.501940,24.137931,17.153946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,DaMarcus Beasley,1030,113.679612,5,15,1.310680,33.333333,10,17,1.485437,58.823529,2.271845,15.384615,8.300971
56,Cristian Cásseres Jr.,1415,96.296820,14,47,2.989399,29.787234,6,30,1.908127,20.000000,1.971731,19.354839,22.325088
284,Yuya Kubo,796,146.871859,8,28,3.165829,28.571429,9,26,2.939698,34.615385,1.469849,15.384615,20.012563
91,Fabian Herbers,802,120.860349,3,11,1.234414,27.272727,1,15,1.683292,6.666667,2.132170,26.315789,16.384040


In [445]:
# Get the percetile scores for all of these stats
AllYears_target['PrgDist_p90_perc'] = AllYears_target['PrgDist_p90'].apply(lambda x: stats.percentileofscore(AllYears_target['PrgDist_p90'], x, 'mean'))
AllYears_target['Possession_Dribbles_Succ%_perc'] = AllYears_target['Possession_Dribbles_Succ%'].apply(lambda x: stats.percentileofscore(AllYears_target['Possession_Dribbles_Succ%'], x, 'mean'))
AllYears_target['Defensive Actions_Vs Dribbles_Tkl%_perc'] = AllYears_target['Defensive Actions_Vs Dribbles_Tkl%'].apply(lambda x: stats.percentileofscore(AllYears_target['Defensive Actions_Vs Dribbles_Tkl%'], x, 'mean'))
AllYears_target['CrsPA%_perc'] = AllYears_target['CrsPA%'].apply(lambda x: stats.percentileofscore(AllYears_target['CrsPA%'], x, 'mean'))
AllYears_target['Press_p90_perc'] = AllYears_target['Press_p90'].apply(lambda x: stats.percentileofscore(AllYears_target['Press_p90'], x, 'mean'))

AllYears_target['PercentileAvg'] = (AllYears_target['PrgDist_p90_perc'] + AllYears_target['Possession_Dribbles_Succ%_perc'] + AllYears_target['Defensive Actions_Vs Dribbles_Tkl%_perc'] + AllYears_target['CrsPA%_perc'] + AllYears_target['Press_p90_perc'])/5
AllYears_target.sort_values(by='PercentileAvg', ascending=False)

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90,PrgDist_p90_perc,Possession_Dribbles_Succ%_perc,Defensive Actions_Vs Dribbles_Tkl%_perc,CrsPA%_perc,Press_p90_perc,PercentileAvg
147,Jorge Moreira,2319,217.335058,31,39,1.513583,79.487179,39,81,3.143596,48.148148,4.501940,24.137931,17.153946,99.087591,98.540146,83.029197,76.824818,67.700730,85.036496
239,Russell Canouse,750,101.280000,4,6,0.720000,66.666667,12,22,2.640000,54.545455,0.600000,40.000000,20.880000,35.583942,79.927007,94.160584,99.817518,88.868613,79.671533
95,Franco Escobar,2282,117.567923,37,54,2.129711,68.518519,34,76,2.997371,44.736842,1.498685,31.578947,18.378615,57.481752,85.948905,75.000000,96.167883,74.270073,77.773723
156,Justin Che,1015,124.758621,15,25,2.216749,60.000000,18,36,3.192118,50.000000,2.748768,32.258065,17.113300,68.795620,58.576642,87.408759,96.532847,66.605839,75.583942
222,Paxton Pomykal,1422,124.240506,16,20,1.265823,80.000000,19,38,2.405063,50.000000,2.341772,10.810811,21.012658,66.605839,99.087591,87.408759,14.781022,89.598540,71.496350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,Brian Rodríguez,2395,130.697286,98,181,6.801670,54.143646,3,36,1.352818,8.333333,3.118998,7.228916,12.175365,75.729927,37.043796,2.372263,3.832117,19.525547,27.700730
79,Dylan Nealis,1125,114.640000,5,14,1.120000,35.714286,3,23,1.840000,13.043478,1.840000,8.695652,17.120000,53.832117,3.467153,4.927007,8.394161,66.970803,27.518248
24,Ariel Lassiter,878,120.239180,8,22,2.255125,36.363636,2,6,0.615034,33.333333,2.357631,13.043478,9.430524,60.401460,4.197080,46.350365,22.080292,2.372263,27.080292
126,Jeremy Ebobisse,1875,50.736000,16,26,1.248000,61.538462,8,27,1.296000,29.629630,0.960000,10.000000,10.752000,1.642336,63.868613,35.401460,13.138686,7.116788,24.233577


In [446]:
# Focus on 2021
Wings2021 = Wing_PlayerGrouped[Wing_PlayerGrouped['Season'] == 2021]
Wings2021

,Player,Season,Date,#,Age,Min,Sub,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%
5,Aaron Herrera,2021,545691450,594,652.919444,2347,0,1,8,0,0,19,5,9,1,416.0,390,61,48,59,0.9,0.9,3.8,67,11,1118,1455,2056.1,122,1105,143,31,65,1118,1455,2056.1,23744,9184,405,448,2427.6,524,650,2160.5,178,311,1455.1,8,3.8,35,93,45,23,122,1191,264,20,1,191,59,89,0,0,0,0,819,276,360,116,1033,48,244,4,1118,3,23,49,45,61,38,35,17,9,27,57,1010.0,30,390,113,794.6,207,126,57,59,15,0,44,48,109,67,1,1789,120,587,804,535,36,1530,31,65,1173.4,34,5,1105,6596,3614,143,54,16,33,35,1131,1030,2439.2,69,9,1,1,45,15,1,89,48,38,0.0,1.0,0,231,24,25,930.7
21,Adam Lundqvist,2021,505268423,75,683

In [475]:
# Drop players that did not play on the wing in 2021
# This prevents players that retired, moved leagues, or had a major position change after 2020
to_drop = []
for i in Wing_PlayerGrouped.index:
    tmp = Wing_PlayerGrouped[(Wing_PlayerGrouped['Player'] == Wing_PlayerGrouped['Player'][i]) & (Wing_PlayerGrouped['Season'] == 2021)]
    if (len(tmp) < 1):
        to_drop.append(i)
Wings = Wing_PlayerGrouped.drop(index=to_drop)
Wings = Wings.groupby(by='Player').sum().reset_index()
Wings

,Player,Season,Date,#,Age,Min,Sub,Summary_Performance_Gls,Summary_Performance_Ast,Summary_Performance_PK,Summary_Performance_PKatt,Summary_Performance_Sh,Summary_Performance_SoT,Summary_Performance_CrdY,Summary_Performance_CrdR,Summary_Performance_Touches,Summary_Performance_Press,Summary_Performance_Tkl,Summary_Performance_Int,Summary_Performance_Blocks,Summary_Expected_xG,Summary_Expected_npxG,Summary_Expected_xA,Summary_SCA_SCA,Summary_SCA_GCA,Summary_Passes_Cmp,Summary_Passes_Att,Summary_Passes_Cmp%,Summary_Passes_Prog,Summary_Carries_Carries,Summary_Carries_Prog,Summary_Dribbles_Succ,Summary_Dribbles_Att,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_ProgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing__Ast,Passing__xA,Passing__KP,Passing__1/3,Passing__PPA,Passing__CrsPA,Passing__Prog,Pass Types_Pass Types_Live,Pass Types_Pass Types_Dead,Pass Types_Pass Types_FK,Pass Types_Pass Types_TB,Pass Types_Pass Types_Press,Pass Types_Pass Types_Sw,Pass Types_Pass Types_Crs,Pass Types_Pass Types_CK,Pass Types_Corner Kicks_In,Pass Types_Corner Kicks_Out,Pass Types_Corner Kicks_Str,Pass Types_Height_Ground,Pass Types_Height_Low,Pass Types_Height_High,Pass Types_Body Parts_Left,Pass Types_Body Parts_Right,Pass Types_Body Parts_Head,Pass Types_Body Parts_TI,Pass Types_Body Parts_Other,Pass Types_Outcomes_Cmp,Pass Types_Outcomes_Off,Pass Types_Outcomes_Out,Pass Types_Outcomes_Int,Pass Types_Outcomes_Blocks,Defensive Actions_Tackles_Tkl,Defensive Actions_Tackles_TklW,Defensive Actions_Tackles_Def 3rd,Defensive Actions_Tackles_Mid 3rd,Defensive Actions_Tackles_Att 3rd,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,Defensive Actions_Vs Dribbles_Tkl%,Defensive Actions_Vs Dribbles_Past,Defensive Actions_Pressures_Press,Defensive Actions_Pressures_Succ,Defensive Actions_Pressures_%,Defensive Actions_Pressures_Def 3rd,Defensive Actions_Pressures_Mid 3rd,Defensive Actions_Pressures_Att 3rd,Defensive Actions_Blocks_Blocks,Defensive Actions_Blocks_Sh,Defensive Actions_Blocks_ShSv,Defensive Actions_Blocks_Pass,Defensive Actions__Int,Defensive Actions__Tkl+Int,Defensive Actions__Clr,Defensive Actions__Err,Possession_Touches_Touches,Possession_Touches_Def Pen,Possession_Touches_Def 3rd,Possession_Touches_Med 3rd,Possession_Touches_Att 3rd,Possession_Touches_Att Pen,Possession_Touches_Live,Possession_Dribbles_Succ,Possession_Dribbles_Att,Possession_Dribbles_Succ%,Possession_Dribbles_#Pl,Possession_Dribbles_Megs,Possession_Carries_Carries,Possession_Carries_TotDist,Possession_Carries_PrgDist,Possession_Carries_Prog,Possession_Carries_1/3,Possession_Carries_CPA,Possession_Carries_Mis,Possession_Carries_Dis,Possession_Receiving_Targ,Possession_Receiving_Rec,Possession_Receiving_Rec%,Possession_Receiving_Prog,Miscellaneous_Performance_CrdY,Miscellaneous_Performance_CrdR,Miscellaneous_Performance_2CrdY,Miscellaneous_Performance_Fls,Miscellaneous_Performance_Fld,Miscellaneous_Performance_Off,Miscellaneous_Performance_Crs,Miscellaneous_Performance_Int,Miscellaneous_Performance_TklW,Miscellaneous_Performance_PKwon,Miscellaneous_Performance_PKcon,Miscellaneous_Performance_OG,Miscellaneous_Performance_Recov,Miscellaneous_Aerial Duels_Won,Miscellaneous_Aerial Duels_Lost,Miscellaneous_Aerial Duels_Won%
0,Aaron Herrera,6060,1656391459,1804,1890.930556,7180,1,1,12,0,0,55,13,23,2,416.0,1063,181,128,183,2.1,2.1,7.3,159,18,3370,4310,6392.8,310,3268,415,113,185,3370,4310,6392.8,72393,26113,1192,1333,7319.2,1571,1876,6833.0,582,978,4815.3,12,7.3,80,248,86,43,310,3508,802,88,6,613,231,195,2,0,0,2,2440,832,1038,428,3002,121,711,12,3370,15,69,95,133,181,111,111,53,17,85,167,3398.3,82,1063,322,2617.6,559,364,140,183,43,1,140,128,309,218,1,5289,362,1809,2479,1424,85,4503,113,185,3908.3,119,9,3268,18835,10254,415,165,37,70,106,3179,2929,7500.6,169,23,2,1,112,48,3,195,128,111,0.0,2.0,1

In [477]:
# Offensive Progressive Dribbling Ability
Wings['PrgDist_p90'] = Wings['Possession_Carries_PrgDist']/(Wings['Min']/90)

# Offensive 1v1 Ability
Wings['Possession_Dribbles_Succ%'] = (Wings['Possession_Dribbles_Succ'] / Wings['Possession_Dribbles_Att'])*100
Wings['Dribbles_Att_p90'] = Wings['Possession_Dribbles_Att']/(Wings['Min']/90)

# Defensive 1v1 Ability
Wings['Defensive Actions_Vs Dribbles_Tkl%'] = (Wings['Defensive Actions_Vs Dribbles_Tkl'] / Wings['Defensive Actions_Vs Dribbles_Att'])*100
Wings['1v1_Tkls_p90'] = Wings['Defensive Actions_Vs Dribbles_Att']/(Wings['Min']/90)

# Crossing Ability
Wings['CrsPA%'] = (Wings['Passing__CrsPA'] / Wings['Pass Types_Pass Types_Crs'])*100
Wings['Crs_p90'] = Wings['Pass Types_Pass Types_Crs']/(Wings['Min']/90)

# Fitness
Wings['Press_p90'] = Wings['Defensive Actions_Pressures_Press']/(Wings['Min']/90)

target_cols = ['Player', 'Min', 'PrgDist_p90', 'Possession_Dribbles_Succ', 'Possession_Dribbles_Att', 'Dribbles_Att_p90', 'Possession_Dribbles_Succ%',
               'Defensive Actions_Vs Dribbles_Tkl', 'Defensive Actions_Vs Dribbles_Att', '1v1_Tkls_p90', 'Defensive Actions_Vs Dribbles_Tkl%',
               'Crs_p90', 'CrsPA%', 'Press_p90']
Wings_target = Wings[(Wings['Dribbles_Att_p90'] >= 0.5) & (Wings['1v1_Tkls_p90'] >= 0.5) & (Wings['Crs_p90'] >= 0.5)].sort_values(by='Possession_Dribbles_Succ%', ascending=False)[target_cols]

Wings_target['PrgDist_p90_perc'] = Wings_target['PrgDist_p90'].apply(lambda x: stats.percentileofscore(Wings_target['PrgDist_p90'], x, 'mean'))
Wings_target['Possession_Dribbles_Succ%_perc'] = Wings_target['Possession_Dribbles_Succ%'].apply(lambda x: stats.percentileofscore(Wings_target['Possession_Dribbles_Succ%'], x, 'mean'))
Wings_target['Defensive Actions_Vs Dribbles_Tkl%_perc'] = Wings_target['Defensive Actions_Vs Dribbles_Tkl%'].apply(lambda x: stats.percentileofscore(Wings_target['Defensive Actions_Vs Dribbles_Tkl%'], x, 'mean'))
Wings_target['CrsPA%_perc'] = Wings_target['CrsPA%'].apply(lambda x: stats.percentileofscore(Wings_target['CrsPA%'], x, 'mean'))
Wings_target['Press_p90_perc'] = Wings_target['Press_p90'].apply(lambda x: stats.percentileofscore(Wings_target['Press_p90'], x, 'mean'))

### Running the same thing, but comparing normalized stats instead of percentiles
#### Normalizing values from 0-1 using x_norm = (x-x_min)/(x_max-x_min)
#### This preserves the player's relative ability in the statistic

In [479]:
Wings_target['PrgDist_p90_norm'] = (Wings_target['PrgDist_p90']-Wings_target['PrgDist_p90'].min())/(Wings_target['PrgDist_p90'].max()-Wings_target['PrgDist_p90'].min())
Wings_target['Possession_Dribbles_Succ%_norm'] = (Wings_target['Possession_Dribbles_Succ%']-Wings_target['Possession_Dribbles_Succ%'].min())/(Wings_target['Possession_Dribbles_Succ%'].max()-Wings_target['Possession_Dribbles_Succ%'].min())
Wings_target['Defensive Actions_Vs Dribbles_Tkl%_norm'] = (Wings_target['Defensive Actions_Vs Dribbles_Tkl%']-Wings_target['Defensive Actions_Vs Dribbles_Tkl%'].min())/(Wings_target['Defensive Actions_Vs Dribbles_Tkl%'].max()-Wings_target['Defensive Actions_Vs Dribbles_Tkl%'].min())
Wings_target['CrsPA%_norm'] = (Wings_target['CrsPA%']-Wings_target['CrsPA%'].min())/(Wings_target['CrsPA%'].max()-Wings_target['CrsPA%'].min())
Wings_target['Press_p90_norm'] = (Wings_target['Press_p90']-Wings_target['Press_p90'].min())/(Wings_target['Press_p90'].max()-Wings_target['Press_p90'].min())
Wings_target[['Player', 'PrgDist_p90_norm', 'Possession_Dribbles_Succ%_norm', 'Defensive Actions_Vs Dribbles_Tkl%_norm', 'CrsPA%_norm', 'Press_p90_norm']]

,Player,PrgDist_p90_norm,Possession_Dribbles_Succ%_norm,Defensive Actions_Vs Dribbles_Tkl%_norm,CrsPA%_norm,Press_p90_norm
75,Javain Brown,0.206828,1.000000,0.568015,0.509091,0.432860
52,Emmanuel Más,0.517898,0.991667,0.572368,0.329412,0.202843
141,Paxton Pomykal,0.515179,0.963333,0.609375,0.302703,0.555054
51,Ema Twumasi,0.633678,0.955342,0.609375,0.386207,0.266845
29,Chase Gasper,0.423074,0.906362,0.609375,0.494118,0.222673
...,...,...,...,...,...,...
70,Jacob Shaffelburg,0.414911,0.273214,0.207589,0.518519,0.463163
74,Jared Stroud,0.396353,0.236111,0.328125,0.400000,0.576425
33,Cristian Cásseres Jr.,0.348403,0.180851,0.187500,0.541935,0.613034
56,Fabian Herbers,0.495005,0.141667,0.000000,0.736842,0.350571


In [480]:
# Add in versatility data - % of minutes at secondary positions
# Additional fitness measure - % of games started without substitute
# Ground covered per game (per 90) would be good, but we don't have tracking data
Wings_target['Secondary_Mins'] = np.nan
Wings_target['Games_Finished'] = np.nan
for i in Wings_target.index:
    player_df = PlayerGameData[PlayerGameData['Player'] == Wings_target['Player'][i]]
    min_df = player_df.groupby('Pos').sum().sort_values(by='Min', ascending=False)['Min'].reset_index()
    if (len(min_df) == 1):
        secondary_mins = 0
    else:
        secondary_mins = (sum(min_df['Min'][1:])/sum(min_df['Min']))*100
    Wings_target['Secondary_Mins'][i] = secondary_mins
    # Games_finished
    starts = player_df[player_df['Sub'] == 0]
    Wings_target['Games_Finished'][i] = pd.isna(starts['Replace']).mean()*100
    
    
Wings_target['Secondary_Mins_norm'] = (Wings_target['Secondary_Mins']-Wings_target['Secondary_Mins'].min())/(Wings_target['Secondary_Mins'].max()-Wings_target['Secondary_Mins'].min())
Wings_target['Games_Finished_norm'] = (Wings_target['Games_Finished']-Wings_target['Games_Finished'].min())/(Wings_target['Games_Finished'].max()-Wings_target['Games_Finished'].min())
#Wings2021_target.sort_values(by='Secondary_Mins')[['Player', 'Min', 'Secondary_Mins']]

In [482]:
# Custom weighting - easy to adjust to increase importance of certain areas
# fitness and versatility are halved because their metrics' validity is uncertain
att_drib_total_w = 1
def_drib_total_w = 1
crossing_total_w = 1
fitness_total_w = 0.5
versatility_total_w = 0.5

PrgDist_p90_w = att_drib_total_w/2
Att_Dribble_w = att_drib_total_w/2
Def_Dribble_w = def_drib_total_w
Crossing_w = crossing_total_w
Pressing_w = fitness_total_w/2
SecondaryMins_w = versatility_total_w
GamesFinished_w = fitness_total_w/2
#Wings_target['PrgDist_p90_norm']*PrgDist_p90_w ---- removed from weighted normavg

Wings_target['Weighted_NormAvg'] = (Wings_target['Possession_Dribbles_Succ%_norm']*Att_Dribble_w + \
                                    Wings_target['Defensive Actions_Vs Dribbles_Tkl%_norm']*Def_Dribble_w + \
                                    Wings_target['CrsPA%_norm']*Crossing_w + \
                                    Wings_target['Press_p90_norm']*Pressing_w + \
                                    Wings_target['Secondary_Mins_norm']*SecondaryMins_w + \
                                    Wings_target['Games_Finished_norm']*GamesFinished_w) / \
                                    (Att_Dribble_w + Def_Dribble_w + Crossing_w + Pressing_w + SecondaryMins_w + GamesFinished_w)
Targets = Wings_target.sort_values(by='Weighted_NormAvg', ascending=False).head(20)
Targets

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90,PrgDist_p90_perc,Possession_Dribbles_Succ%_perc,Defensive Actions_Vs Dribbles_Tkl%_perc,CrsPA%_perc,Press_p90_perc,PrgDist_p90_norm,Possession_Dribbles_Succ%_norm,Defensive Actions_Vs Dribbles_Tkl%_norm,CrsPA%_norm,Press_p90_norm,Secondary_Mins,Games_Finished,Secondary_Mins_norm,Games_Finished_norm,Weighted_NormAvg
152,Russell Teibert,1019,63.591757,10,17,1.501472,58.823529,10,20,1.766438,50.000000,1.236506,35.714286,20.314033,3.216374,59.064327,87.426901,99.707602,88.596491,0.153209,0.633333,0.609375,1.000000,0.524190,48.932161,61.842105,0.603222,0.627651,0.718747
65,Hassani Dotson,2320,104.974138,39,59,2.288793,66.101695,19,56,2.172414,33.928571,1.474138,28.947368,22.965517,39.473684,84.502924,40.643275,95.614035,94.444444,0.400192,0.746751,0.383371,0.810526,0.641327,76.256212,91.228070,0.947279,0.925897,0.695062
15,Boris Sekulić,3768,80.708599,33,53,1.265924,62.264151,32,55,1.313694,58.181818,2.173567,30.769231,9.506369,9.064327,74.853801,96.198830,97.368421,2.631579,0.255367,0.686950,0.724432,0.861538,0.046729,41.773685,70.833333,0.513084,0.718905,0.679256
77,Jaylin Lindsey,2176,106.668199,16,30,1.240809,53.333333,24,32,1.323529,75.000000,2.192096,24.528302,12.366728,42.982456,38.304094,99.122807,80.409357,18.421053,0.410302,0.547778,0.960937,0.686792,0.173094,20.315928,95.652174,0.242894,0.970798,0.665440
144,Raheem Edwards,1113,93.396226,35,56,4.528302,62.500000,10,37,2.991914,27.027027,1.617251,35.000000,25.714286,23.099415,75.730994,26.023392,99.122807,97.368421,0.331091,0.690625,0.286318,0.980000,0.762762,71.923475,31.578947,0.892723,0.320503,0.665374
90,Joseph Mora,4538,92.915381,45,73,1.447774,61.643836,53,113,2.241075,46.902655,1.665932,29.761905,16.183341,21.345029,70.467836,75.730994,96.198830,57.017544,0.328221,0.677283,0.565819,0.833333,0.341704,47.694555,68.518519,0.587638,0.695412,0.654540
111,Kyle Smith,4270,94.067916,31,53,1.117096,58.490566,60,132,2.782201,45.454545,1.622951,24.675325,17.367681,24.269006,57.602339,73.976608,80.994152,69.298246,0.335100,0.628145,0.545455,0.690909,0.394026,69.723105,75.510204,0.865016,0.766372,0.649441
35,Cristian Roldan,4149,99.436009,67,99,2.147505,67.676768,25,79,1.713666,31.645570,2.386117,23.636364,16.876356,28.362573,89.181287,36.549708,78.070175,64.035088,0.367138,0.771296,0.351266,0.661818,0.372320,80.255682,93.103448,0.997639,0.944931,0.636247
21,Brooks Lennon,6008,115.436085,71,105,1.572903,67.619048,40,115,1.722703,34.782609,3.834887,26.562500,14.545606,55.847953,88.596491,45.029240,90.350877,45.321637,0.462632,0.770397,0.395380,0.743750,0.269353,61.282306,86.363636,0.758732,0.876526,0.625761
123,Marco Farfan,2043,81.189427,22,43,1.894273,51.162791,40,65,2.863436,61.538462,1.277533,20.689655,19.118943,10.818713,30.116959,97.953216,61.403509,83.918129,0.258237,0.513953,0.771635,0.579310,0.471393,53.715499,51.515152,0.663452,0.522840,0.625202


#### Take the 20 best, and get the injury data
#### This will be the cumulative injury minutes in the past 2 years
#### Also check for a recent move

In [484]:
Targets = Wings_target.sort_values(by='Weighted_NormAvg', ascending=False).head(20)
Targets

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90,PrgDist_p90_perc,Possession_Dribbles_Succ%_perc,Defensive Actions_Vs Dribbles_Tkl%_perc,CrsPA%_perc,Press_p90_perc,PrgDist_p90_norm,Possession_Dribbles_Succ%_norm,Defensive Actions_Vs Dribbles_Tkl%_norm,CrsPA%_norm,Press_p90_norm,Secondary_Mins,Games_Finished,Secondary_Mins_norm,Games_Finished_norm,Weighted_NormAvg
152,Russell Teibert,1019,63.591757,10,17,1.501472,58.823529,10,20,1.766438,50.000000,1.236506,35.714286,20.314033,3.216374,59.064327,87.426901,99.707602,88.596491,0.153209,0.633333,0.609375,1.000000,0.524190,48.932161,61.842105,0.603222,0.627651,0.718747
65,Hassani Dotson,2320,104.974138,39,59,2.288793,66.101695,19,56,2.172414,33.928571,1.474138,28.947368,22.965517,39.473684,84.502924,40.643275,95.614035,94.444444,0.400192,0.746751,0.383371,0.810526,0.641327,76.256212,91.228070,0.947279,0.925897,0.695062
15,Boris Sekulić,3768,80.708599,33,53,1.265924,62.264151,32,55,1.313694,58.181818,2.173567,30.769231,9.506369,9.064327,74.853801,96.198830,97.368421,2.631579,0.255367,0.686950,0.724432,0.861538,0.046729,41.773685,70.833333,0.513084,0.718905,0.679256
77,Jaylin Lindsey,2176,106.668199,16,30,1.240809,53.333333,24,32,1.323529,75.000000,2.192096,24.528302,12.366728,42.982456,38.304094,99.122807,80.409357,18.421053,0.410302,0.547778,0.960937,0.686792,0.173094,20.315928,95.652174,0.242894,0.970798,0.665440
144,Raheem Edwards,1113,93.396226,35,56,4.528302,62.500000,10,37,2.991914,27.027027,1.617251,35.000000,25.714286,23.099415,75.730994,26.023392,99.122807,97.368421,0.331091,0.690625,0.286318,0.980000,0.762762,71.923475,31.578947,0.892723,0.320503,0.665374
90,Joseph Mora,4538,92.915381,45,73,1.447774,61.643836,53,113,2.241075,46.902655,1.665932,29.761905,16.183341,21.345029,70.467836,75.730994,96.198830,57.017544,0.328221,0.677283,0.565819,0.833333,0.341704,47.694555,68.518519,0.587638,0.695412,0.654540
111,Kyle Smith,4270,94.067916,31,53,1.117096,58.490566,60,132,2.782201,45.454545,1.622951,24.675325,17.367681,24.269006,57.602339,73.976608,80.994152,69.298246,0.335100,0.628145,0.545455,0.690909,0.394026,69.723105,75.510204,0.865016,0.766372,0.649441
35,Cristian Roldan,4149,99.436009,67,99,2.147505,67.676768,25,79,1.713666,31.645570,2.386117,23.636364,16.876356,28.362573,89.181287,36.549708,78.070175,64.035088,0.367138,0.771296,0.351266,0.661818,0.372320,80.255682,93.103448,0.997639,0.944931,0.636247
21,Brooks Lennon,6008,115.436085,71,105,1.572903,67.619048,40,115,1.722703,34.782609,3.834887,26.562500,14.545606,55.847953,88.596491,45.029240,90.350877,45.321637,0.462632,0.770397,0.395380,0.743750,0.269353,61.282306,86.363636,0.758732,0.876526,0.625761
123,Marco Farfan,2043,81.189427,22,43,1.894273,51.162791,40,65,2.863436,61.538462,1.277533,20.689655,19.118943,10.818713,30.116959,97.953216,61.403509,83.918129,0.258237,0.513953,0.771635,0.579310,0.471393,53.715499,51.515152,0.663452,0.522840,0.625202


In [485]:
# verletzungen - All read by hand from Transfermarkt
injuries_dict = {'Hassani Dotson':{'days_injured':0, 'last_move':20190111},
                 'Ryan Hollingshead':{'days_injured':0, 'last_move':20130117},
                 'Kyle Smith':{'days_injured':0, 'last_move':20181219},
                 'Pedro Santos':{'days_injured':1, 'last_move':20170808},
                 'Russell Teibert':{'days_injured':16, 'last_move':20090701},
                 'Boris Sekulić':{'days_injured':0, 'last_move':20200215},
                 'Auro':{'days_injured':22, 'last_move':20180213},
                 'Kyle Duncan':{'days_injured':0, 'last_move':20220101},
                 'Andrew Gutman':{'days_injured':0, 'last_move':20210309},
                 'Andre Shinyashiki':{'days_injured':0, 'last_move':20190111},
                 'Jaylin Lindsey':{'days_injured':0, 'last_move':20211221},
                 'Khiry Shelton':{'days_injured':0, 'last_move':20171214},
                 'Raheem Edwards':{'days_injured':0, 'last_move':20220107},
                 'Julian Araujo':{'days_injured':0, 'last_move':20180101},
                 'Justin Che':{'days_injured':21, 'last_move':20200725},
                 'John Tolkin':{'days_injured':0, 'last_move':20190704},
                 'Cole Bassett':{'days_injured':0, 'last_move':20170901},
                 'Brooks Lennon':{'days_injured':0, 'last_move':20191202},
                 'Przemysław Frankowski':{'days_injured':0, 'last_move':20210805},
                 'Aaron Herrera':{'days_injured':0, 'last_move':20150801},
                 'Joseph Mora':{'days_injured':6, 'last_move':20211214},
                 'Joe Gyau':{'days_injured':67, 'last_move':20190808},
                 'Cristian Roldan':{'days_injured':0, 'last_move':20150115},
                 'Jhon Espinoza':{'days_injured':0, 'last_move':20210101},
                 'Paxton Pomykal':{'days_injured':97+15, 'last_move':20140701},
                 'Jesus Ferreira':{'days_injured':65, 'last_move':20170101},
                 'Ema Twumasi':{'days_injured':0, 'last_move':20201020},
                 'Maximiliano Moralez':{'days_injured':17+32, 'last_move':20170215},
                 'Marco Farfan':{'days_injured':0, 'last_move':20210101},
                 'Javain Brown':{'days_injured':0, 'last_move':20210121},
                 'Sam Junqua':{'days_injured':75, 'last_move':20190111},
                 'Latif Blessing':{'days_injured':0, 'last_move':20171212},
                 'Kemar Lawrence':{'days_injured':34+5, 'last_move':20210507},
                 'Djordje Mihailovic':{'days_injured':0, 'last_move':20201217},
                 'Yimmi Chará':{'days_injured':0, 'last_move':20200102},
                 'João Moutinho':{'days_injured':21+46+235, 'last_move':20181211}}

Targets['Days_Injured'] = np.nan
Targets['Last_Move'] = np.nan
for i in Targets.index:
    Targets['Days_Injured'][i] = injuries_dict[Targets['Player'][i]]['days_injured']
    Targets['Last_Move'][i] = injuries_dict[Targets['Player'][i]]['last_move']
    
Targets = Targets[Targets['Last_Move'] < 20210701] # Players that haven't moved since July 2021
Targets

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90,PrgDist_p90_perc,Possession_Dribbles_Succ%_perc,Defensive Actions_Vs Dribbles_Tkl%_perc,CrsPA%_perc,Press_p90_perc,PrgDist_p90_norm,Possession_Dribbles_Succ%_norm,Defensive Actions_Vs Dribbles_Tkl%_norm,CrsPA%_norm,Press_p90_norm,Secondary_Mins,Games_Finished,Secondary_Mins_norm,Games_Finished_norm,Weighted_NormAvg,Days_Injured,Last_Move
152,Russell Teibert,1019,63.591757,10,17,1.501472,58.823529,10,20,1.766438,50.000000,1.236506,35.714286,20.314033,3.216374,59.064327,87.426901,99.707602,88.596491,0.153209,0.633333,0.609375,1.000000,0.524190,48.932161,61.842105,0.603222,0.627651,0.718747,16.0,20090701.0
65,Hassani Dotson,2320,104.974138,39,59,2.288793,66.101695,19,56,2.172414,33.928571,1.474138,28.947368,22.965517,39.473684,84.502924,40.643275,95.614035,94.444444,0.400192,0.746751,0.383371,0.810526,0.641327,76.256212,91.228070,0.947279,0.925897,0.695062,0.0,20190111.0
15,Boris Sekulić,3768,80.708599,33,53,1.265924,62.264151,32,55,1.313694,58.181818,2.173567,30.769231,9.506369,9.064327,74.853801,96.198830,97.368421,2.631579,0.255367,0.686950,0.724432,0.861538,0.046729,41.773685,70.833333,0.513084,0.718905,0.679256,0.0,20200215.0
111,Kyle Smith,4270,94.067916,31,53,1.117096,58.490566,60,132,2.782201,45.454545,1.622951,24.675325,17.367681,24.269006,57.602339,73.976608,80.994152,69.298246,0.335100,0.628145,0.545455,0.690909,0.394026,69.723105,75.510204,0.865016,0.766372,0.649441,0.0,20181219.0
35,Cristian Roldan,4149,99.436009,67,99,2.147505,67.676768,25,79,1.713666,31.645570,2.386117,23.636364,16.876356,28.362573,89.181287,36.549708,78.070175,64.035088,0.367138,0.771296,0.351266,0.661818,0.372320,80.255682,93.103448,0.997639,0.944931,0.636247,0.0,20150115.0
21,Brooks Lennon,6008,115.436085,71,105,1.572903,67.619048,40,115,1.722703,34.782609,3.834887,26.562500,14.545606,55.847953,88.596491,45.029240,90.350877,45.321637,0.462632,0.770397,0.395380,0.743750,0.269353,61.282306,86.363636,0.758732,0.876526,0.625761,0.0,20191202.0
123,Marco Farfan,2043,81.189427,22,43,1.894273,51.162791,40,65,2.863436,61.538462,1.277533,20.689655,19.118943,10.818713,30.116959,97.953216,61.403509,83.918129,0.258237,0.513953,0.771635,0.579310,0.471393,53.715499,51.515152,0.663452,0.522840,0.625202,0.0,20210101.0
96,Justin Che,1015,124.758621,15,25,2.216749,60.000000,18,36,3.192118,50.000000,2.748768,32.258065,17.113300,70.467836,64.327485,87.426901,97.953216,67.543860,0.518271,0.651667,0.609375,0.903226,0.382788,12.598425,66.666667,0.145717,0.676617,0.621755,21.0,20200725.0
80,Jhon Espinoza,813,116.789668,11,22,2.435424,50.000000,13,24,2.656827,54.166667,3.874539,25.714286,13.505535,58.187135,26.023392,94.444444,85.672515,34.795322,0.470710,0.495833,0.667969,0.720000,0.223404,59.040590,37.500000,0.730505,0.380597,0.614897,0.0,20210101.0
84,John Tolkin,1777,116.539111,15,27,1.367473,55.555556,14,35,1.772651,40.000000,1.924592,26.315789,17.017445,57.602339,46.198830,61.111111,88.011696,65.789474,0.469215,0.582407,0.468750,0.736842,0.378553,58.940069,72.727273,0.729239,0.738128,0.611596,0.0,20190704.0


In [486]:
Targets['Age'] = np.nan
Targets['Age'] = Targets['Player'].apply(lambda x: math.floor(PlayerGameData[PlayerGameData['Player'] == x].sort_values(by='Date', ascending=False)['Age'].iloc[0]))
Targets = Targets[Targets['Age'] <= 30] # Wingers peak early and decline quickly
Targets[['Player', 'Age']]

,Player,Age
152,Russell Teibert,28
65,Hassani Dotson,24
15,Boris Sekulić,30
111,Kyle Smith,29
35,Cristian Roldan,26
21,Brooks Lennon,24
123,Marco Farfan,22
96,Justin Che,17
80,Jhon Espinoza,22
84,John Tolkin,19


In [487]:
Targets

,Player,Min,PrgDist_p90,Possession_Dribbles_Succ,Possession_Dribbles_Att,Dribbles_Att_p90,Possession_Dribbles_Succ%,Defensive Actions_Vs Dribbles_Tkl,Defensive Actions_Vs Dribbles_Att,1v1_Tkls_p90,Defensive Actions_Vs Dribbles_Tkl%,Crs_p90,CrsPA%,Press_p90,PrgDist_p90_perc,Possession_Dribbles_Succ%_perc,Defensive Actions_Vs Dribbles_Tkl%_perc,CrsPA%_perc,Press_p90_perc,PrgDist_p90_norm,Possession_Dribbles_Succ%_norm,Defensive Actions_Vs Dribbles_Tkl%_norm,CrsPA%_norm,Press_p90_norm,Secondary_Mins,Games_Finished,Secondary_Mins_norm,Games_Finished_norm,Weighted_NormAvg,Days_Injured,Last_Move,Age
152,Russell Teibert,1019,63.591757,10,17,1.501472,58.823529,10,20,1.766438,50.000000,1.236506,35.714286,20.314033,3.216374,59.064327,87.426901,99.707602,88.596491,0.153209,0.633333,0.609375,1.000000,0.524190,48.932161,61.842105,0.603222,0.627651,0.718747,16.0,20090701.0,28
65,Hassani Dotson,2320,104.974138,39,59,2.288793,66.101695,19,56,2.172414,33.928571,1.474138,28.947368,22.965517,39.473684,84.502924,40.643275,95.614035,94.444444,0.400192,0.746751,0.383371,0.810526,0.641327,76.256212,91.228070,0.947279,0.925897,0.695062,0.0,20190111.0,24
15,Boris Sekulić,3768,80.708599,33,53,1.265924,62.264151,32,55,1.313694,58.181818,2.173567,30.769231,9.506369,9.064327,74.853801,96.198830,97.368421,2.631579,0.255367,0.686950,0.724432,0.861538,0.046729,41.773685,70.833333,0.513084,0.718905,0.679256,0.0,20200215.0,30
111,Kyle Smith,4270,94.067916,31,53,1.117096,58.490566,60,132,2.782201,45.454545,1.622951,24.675325,17.367681,24.269006,57.602339,73.976608,80.994152,69.298246,0.335100,0.628145,0.545455,0.690909,0.394026,69.723105,75.510204,0.865016,0.766372,0.649441,0.0,20181219.0,29
35,Cristian Roldan,4149,99.436009,67,99,2.147505,67.676768,25,79,1.713666,31.645570,2.386117,23.636364,16.876356,28.362573,89.181287,36.549708,78.070175,64.035088,0.367138,0.771296,0.351266,0.661818,0.372320,80.255682,93.103448,0.997639,0.944931,0.636247,0.0,20150115.0,26
21,Brooks Lennon,6008,115.436085,71,105,1.572903,67.619048,40,115,1.722703,34.782609,3.834887,26.562500,14.545606,55.847953,88.596491,45.029240,90.350877,45.321637,0.462632,0.770397,0.395380,0.743750,0.269353,61.282306,86.363636,0.758732,0.876526,0.625761,0.0,20191202.0,24
123,Marco Farfan,2043,81.189427,22,43,1.894273,51.162791,40,65,2.863436,61.538462,1.277533,20.689655,19.118943,10.818713,30.116959,97.953216,61.403509,83.918129,0.258237,0.513953,0.771635,0.579310,0.471393,53.715499,51.515152,0.663452,0.522840,0.625202,0.0,20210101.0,22
96,Justin Che,1015,124.758621,15,25,2.216749,60.000000,18,36,3.192118,50.000000,2.748768,32.258065,17.113300,70.467836,64.327485,87.426901,97.953216,67.543860,0.518271,0.651667,0.609375,0.903226,0.382788,12.598425,66.666667,0.145717,0.676617,0.621755,21.0,20200725.0,17
80,Jhon Espinoza,813,116.789668,11,22,2.435424,50.000000,13,24,2.656827,54.166667,3.874539,25.714286,13.505535,58.187135,26.023392,94.444444,85.672515,34.795322,0.470710,0.495833,0.667969,0.720000,0.223404,59.040590,37.500000,0.730505,0.380597,0.614897,0.0,20210101.0,22
84,John Tolkin,1777,116.539111,15,27,1.367473,55.555556,14,35,1.772651,40.000000,1.924592,26.315789,17.017445,57.602339,46.198830,61.111111,88.011696,65.789474,0.469215,0.582407,0.468750,0.736842,0.378553,58.940069,72.727273,0.729239,0.738128,0.611596,0.0,20190704.0,19


In [500]:
# Plot them all so you can flip through the identically-structured plots
ctr=0
for player in Targets['Player'].unique():
#for player in ['Ryan Hollingshead']:
    ctr+=1
    #player = 'Auro'
    print(player)
    bound=10
    window=0.02
    alpha=0.2
    n_stats=6

    dotcolor_back='#190F44'
    dotcolor_target='#ECE838'

    for_plot = Wings_target.copy().reset_index(drop=True)

    # Progressive Dribbling - 'PrgDist_p90_norm' - REMOVED
    # Attacking 1v1 - 'Possession_Dribbles_Succ%_norm'
    # Defending 1v1 Dribbling - 'Defensive Actions_Vs Dribbles_Tkl%_norm'
    # Crosses - 'CrsPA%_norm'
    # Pressing Volume - 'Press_p90_norm'
    # Fitness - 'Games_Finished_norm'
    # Versatility - 'Secondary_Mins_norm'

    # 0: {'stat_col':'PrgDist_p90', 'title':'Progressive Carry Distance (per 90)', 'ylabel':'Progressive Carry Distance (per 90) (yards)'},
    
    plot_dict = {0: {'stat_col':'Possession_Dribbles_Succ%', 'title':'1v1 Dribble Success %', 'ylabel':'1v1 Dribble Success %'},
                 1: {'stat_col':'Defensive Actions_Vs Dribbles_Tkl%', 'title':'1v1 Tackle Success %', 'ylabel':'1v1 Tackle Success %'},
                 2: {'stat_col':'CrsPA%', 'title':'Cross Completion %', 'ylabel':'Cross Completion %'},
                 3: {'stat_col':'Press_p90', 'title':'Pressures per 90', 'ylabel':'Pressures per 90'},
                 4: {'stat_col':'Games_Finished', 'title':"% of Starts Finished", 'ylabel':'% of Starts Not Substituted Out'},
                 5: {'stat_col':'Secondary_Mins', 'title':'Versatility', 'ylabel':'% of Minutes Played not at Primary Position'}}


    fig, ax = plt.subplots(1, n_stats, figsize=(16,8))

    ax_i = 0
    for ax_i in range(n_stats):
        stat_col = plot_dict[ax_i]['stat_col']
        stat_sorted = for_plot.sort_values(by=stat_col).reset_index(drop=True)
        player_idx = stat_sorted[stat_sorted['Player'] == player].index[0]
        swarm = sns.swarmplot(y=stat_sorted[stat_col], color=dotcolor_back, alpha=alpha, ax=ax[ax_i], s=7.5, zorder=1)
        xy = swarm.collections[0].get_offsets()
        player_dot = ax[ax_i].scatter(x=[xy[player_idx][0]], y=[xy[player_idx][1]], color=dotcolor_target, edgecolor='black', s=225, zorder=3, label=f"{player}\nvs. Wingers/Backs")
        ax[ax_i].set_xlim(xy[:,0].min()*1.5, xy[:,0].max()*1.5)
        ymin, ymax = ax[ax_i].get_ylim()[0], ax[ax_i].get_ylim()[1]
        if (ax_i == 0):
            ax[ax_i].set_ylim(ymin, ymax+((ymax-ymin)*.05))
        ax[ax_i].set_ylabel(plot_dict[ax_i]['ylabel'], fontsize=16)
        ax[ax_i].set_xticks([])
        if (ax_i%2 == 0):
            title = f"\n{plot_dict[ax_i]['title']}"
        else:
            title = f"{plot_dict[ax_i]['title']}\n"
        ax[ax_i].set_title(title, fontsize=20, clip_on=False)
        if (xy[player_idx][0] >= 0): ha='left'
        else: ha='right'
        t = ax[ax_i].text(x=xy[player_idx][0]*0.9, y=xy[player_idx][1]*1.02,
                   s=f"{stats.percentileofscore(stat_sorted[stat_col], stat_sorted[stat_col][player_idx], 'weak'):.1f}%",
                   fontsize=9, clip_on=False, ha=ha)

        ax[ax_i].spines['top'].set_visible(False)
        #ax[ax_i].spines['bottom'].set_visible(False)
        ax[ax_i].spines['right'].set_visible(False)


    lgnd = ax[0].legend(handles=[player_dot], fontsize=8.5, loc='upper left', handletextpad=0.1)
    lgnd.legendHandles[0]._sizes = [100]
    ax[0].text(x=0.2, y=-0.02, s="Data from StatsBomb", va='top', ha='left',
               transform=ax[0].transAxes, fontsize=14, clip_on=False)
    ax[1].text(x=-0.1, y=-0.02, s="via FBRef.com", va='top', ha='left',
               transform=ax[1].transAxes, fontsize=14, clip_on=False)

    ax[n_stats-1].text(s=f"{player}\nvs. 2021 MLS Wingers/Backs", x=1.1, y=0.5, fontsize=24, rotation=270,
                       transform=ax[n_stats-1].transAxes, va='center', ha='center', clip_on=False)

    #plt.subplots_adjust(top=0.95)
    plt.tight_layout()
    plt.savefig(f"Recruiting_Swarmplots/{str(ctr).zfill(2)}_{player.replace(' ', '-')}_Recruiting_Swarmplot.png", dpi=200, bbox_inches='tight')
    plt.clf()

Russell Teibert
Hassani Dotson
Boris Sekulić
Kyle Smith
Cristian Roldan
Brooks Lennon
Marco Farfan
Justin Che
Jhon Espinoza
John Tolkin
Jesus Ferreira
Javain Brown
Auro
Sam Junqua
Joe Gyau
Paxton Pomykal


<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

...

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

In [458]:
# For reference when identifying roster fit
player_salaries = {'Russell Teibert' : '387,000',
                   'Hassani Dotson' : '81,375',
                   'Kyle Smith' : '118,660',
                   'Cristian Roldan' : '881,542',
                   'Brooks Lennon' : '375,000',
                   'Marco Farfan' : '190,000',
                   'Justin Che' : '89,209',
                   'Jhon Espinoza' : '188,500',
                   'John Tolkin' : '86,375',
                   'Jesus Ferreira' : '550,000',
                   'Javain Brown' : '63,547',
                   'Auro' : '388,750',
                   'Sam Junqua' : '81,375',
                   'Joe Gyau' : '273,000',
                   'Paxton Pomykal' : '600,000'}

In [462]:
# Use this (replace player var) to see how a player's position minutes are distributed
# For example, we see Dotson has enough wing minutes to make the cutoff, but is mostly played at a center mid in Minnesota
player = 'Hassani Dotson'
tmp_df = pd.DataFrame()
tmp_df['Pos'] = PlayerGameData[PlayerGameData['Player'] == player].groupby('Pos').count().index
tmp_df['Games'] = list(PlayerGameData[PlayerGameData['Player'] == player].groupby('Pos').count()['Player'])
tmp_df['Mins'] = list(PlayerGameData[PlayerGameData['Player'] == player].groupby('Pos').sum()['Min'])
tmp_df.sort_values(by='Mins', ascending=False)

,Pos,Games,Mins
2,CM,46,3339
5,"CM,RB",10,900
13,"RB,CM",5,450
9,DM,5,366
18,WB,4,269
12,RB,3,239
6,"CM,RB,RW",2,180
8,"CM,RW",2,180
16,"RB,RW",2,180
10,"DM,CM",2,118
